In [1]:
%pip install pandas sqlalchemy psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from dotenv import load_dotenv
import os
pd.set_option('future.no_silent_downcasting', True)
from sqlalchemy import create_engine, text, inspect

# Load environment variables from the .env file
load_dotenv()

# Access the credentials
host = os.getenv('host')
port = os.getenv('port')
database = os.getenv('dbname')
username = os.getenv('user')
password = os.getenv('password')

# Connection details
engine = create_engine(
    f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}',
    future=True  # Ensures SQLAlchemy behaves like 2.0
)

# List available tables in the database after successful connection
inspector = inspect(engine)
tables = inspector.get_table_names()
print("Tables:", tables)

Tables: ['raw_agricultural_data']


In [3]:
# Connect to the database and run the query
with engine.connect() as connection:
    query = text("SELECT * FROM raw_agricultural_data")
    result = connection.execute(query)  # Execute the query
    df = pd.DataFrame(result.fetchall(), columns=result.keys())  # Convert to DataFrame

# Show the first few rows
df.head()

,id,commodity,classification,grade,sex,market,wholesale,retail,supply_volume,county,date
0,1,Wheat,-,-,-,Kibiok,75.00/Kg,80.00/Kg,450.00,Nandi,2024-03-10
1,2,Wheat,-,-,-,Kabiyet Market,100.00/Kg,130.00/Kg,1000.00,Nandi,2024-03-08
2,3,Wheat,-,-,-,Nakuru Wakulima,65.00/Kg,-,NULL,Nakuru,2024-03-08
3,4,Wheat,-,-,-,Daraja Mbili,60.00/Kg,80.00/Kg,250.00,Kisii,2024-03-08
4,5,Wheat,-,-,-,Maua,77.78/Kg,100.00/Kg,1980.00,Meru,2024-03-07


In [4]:
# get column types, nulls
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              2000 non-null   int64 
 1   commodity       2000 non-null   object
 2   classification  2000 non-null   object
 3   grade           2000 non-null   object
 4   sex             2000 non-null   object
 5   market          2000 non-null   object
 6   wholesale       2000 non-null   object
 7   retail          2000 non-null   object
 8   supply_volume   2000 non-null   object
 9   county          2000 non-null   object
 10  date            2000 non-null   object
dtypes: int64(1), object(10)
memory usage: 172.0+ KB


In [5]:
df.duplicated().sum()  # check for duplicates

np.int64(0)

In [6]:
df.describe()       # basic stats

,id
count,2000.000000
mean,1000.500000
std,577.494589
min,1.000000
25%,500.750000
50%,1000.500000
75%,1500.250000
max,2000.000000


In [7]:
df['classification'].unique() #Get column value options

array(['-', 'Sindano', 'Pishori', 'IRR', 'Large Brown', 'Small Red',
       'Hybrid/ polished', 'Makueni', 'Local-Special', 'White Maize',
       'Mixed-Traditional ', 'Yellow Maize'], dtype=object)

In [8]:
df['grade'].unique() #Get column value options

array(['-'], dtype=object)

In [9]:
df['sex'].unique() #Get column value options

array(['-'], dtype=object)

In [10]:
df['commodity'].unique() #Get column value options

array(['Wheat', 'Red Sorghum', 'Rice', 'Ground Nuts', 'Green Grams',
       'Beans Red Haricot (Wairimu)', 'Dry Maize'], dtype=object)

In [11]:
import numpy as np

# Replace 'NULL' and '-' with NaN in the supply_volume column
df['supply_volume'] = df['supply_volume'].replace({'NULL': np.nan, '-': np.nan})

# Replace '-' with NaN in the grade and sex columns
df['grade'] = df['grade'].replace('-', np.nan)
df['sex'] = df['sex'].replace('-', np.nan)

# Convert the column to numeric (this will convert any non-numeric values to NaN)
df['supply_volume'] = pd.to_numeric(df['supply_volume'], errors='coerce')

# Verify the changes
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2000 non-null   int64  
 1   commodity       2000 non-null   object 
 2   classification  2000 non-null   object 
 3   grade           0 non-null      object 
 4   sex             0 non-null      object 
 5   market          2000 non-null   object 
 6   wholesale       2000 non-null   object 
 7   retail          2000 non-null   object 
 8   supply_volume   1158 non-null   float64
 9   county          2000 non-null   object 
 10  date            2000 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 172.0+ KB


,id,commodity,classification,grade,sex,market,wholesale,retail,supply_volume,county,date
0,1,Wheat,-,NaN,NaN,Kibiok,75.00/Kg,80.00/Kg,450.0,Nandi,2024-03-10
1,2,Wheat,-,NaN,NaN,Kabiyet Market,100.00/Kg,130.00/Kg,1000.0,Nandi,2024-03-08
2,3,Wheat,-,NaN,NaN,Nakuru Wakulima,65.00/Kg,-,NaN,Nakuru,2024-03-08
3,4,Wheat,-,NaN,NaN,Daraja Mbili,60.00/Kg,80.00/Kg,250.0,Kisii,2024-03-08
4,5,Wheat,-,NaN,NaN,Maua,77.78/Kg,100.00/Kg,1980.0,Meru,2024-03-07


In [12]:
# Rename the 'retail' and 'wholesale' columns to 'retail_per_kg' and 'wholesale_per_kg'
df = df.rename(columns={'retail': 'retail_per_kg'})
df = df.rename(columns={'wholesale': 'wholesale_per_kg'})

# Replace any '-' with nan for columns 'retail_per_kg' and 'wholesale_per_kg'
df['retail_per_kg'] = df['retail_per_kg'].replace({' - ': np.nan})
df['wholesale_per_kg'] = df['wholesale_per_kg'].replace({' - ': np.nan})

# Check the updated column names
print(df.columns)
df.head()


Index(['id', 'commodity', 'classification', 'grade', 'sex', 'market',
       'wholesale_per_kg', 'retail_per_kg', 'supply_volume', 'county', 'date'],
      dtype='object')


,id,commodity,classification,grade,sex,market,wholesale_per_kg,retail_per_kg,supply_volume,county,date
0,1,Wheat,-,NaN,NaN,Kibiok,75.00/Kg,80.00/Kg,450.0,Nandi,2024-03-10
1,2,Wheat,-,NaN,NaN,Kabiyet Market,100.00/Kg,130.00/Kg,1000.0,Nandi,2024-03-08
2,3,Wheat,-,NaN,NaN,Nakuru Wakulima,65.00/Kg,NaN,NaN,Nakuru,2024-03-08
3,4,Wheat,-,NaN,NaN,Daraja Mbili,60.00/Kg,80.00/Kg,250.0,Kisii,2024-03-08
4,5,Wheat,-,NaN,NaN,Maua,77.78/Kg,100.00/Kg,1980.0,Meru,2024-03-07


In [13]:
# Remove the 'Kg' and any other non-numeric characters, then convert to float
df['retail_per_kg'] = df['retail_per_kg'].str.replace('/Kg', '').astype(float)
df['wholesale_per_kg'] = df['wholesale_per_kg'].str.replace('/Kg', '').astype(float)

# Check the updated columns
df.head(10)

,id,commodity,classification,grade,sex,market,wholesale_per_kg,retail_per_kg,supply_volume,county,date
0,1,Wheat,-,NaN,NaN,Kibiok,75.00,80.0,450.0,Nandi,2024-03-10
1,2,Wheat,-,NaN,NaN,Kabiyet Market,100.00,130.0,1000.0,Nandi,2024-03-08
2,3,Wheat,-,NaN,NaN,Nakuru Wakulima,65.00,NaN,NaN,Nakuru,2024-03-08
3,4,Wheat,-,NaN,NaN,Daraja Mbili,60.00,80.0,250.0,Kisii,2024-03-08
4,5,Wheat,-,NaN,NaN,Maua,77.78,100.0,1980.0,Meru,2024-03-07
5,6,Wheat,-,NaN,NaN,Nakuru Wakulima,65.00,NaN,NaN,Nakuru,2024-03-07
6,7,Wheat,-,NaN,NaN,Kawangware,80.00,90.0,NaN,Nairobi,2024-03-07
7,8,Wheat,-,NaN,NaN,Nakuru Wakulima,65.00,NaN,990.0,Nakuru,2024-03-06
8,9,Wheat,-,NaN,NaN,Nakuru Wakulima,65.00,NaN,NaN,Nakuru,2024-03-05
9,10,Wheat,-,NaN,NaN,Kabiyet Market,200.00,300.0,1000.0,Nandi,2024-03-05


In [14]:
df['supply_volume'] = df['supply_volume'].astype(float) #Covert supply_volume column values for float
df.head()

,id,commodity,classification,grade,sex,market,wholesale_per_kg,retail_per_kg,supply_volume,county,date
0,1,Wheat,-,NaN,NaN,Kibiok,75.00,80.0,450.0,Nandi,2024-03-10
1,2,Wheat,-,NaN,NaN,Kabiyet Market,100.00,130.0,1000.0,Nandi,2024-03-08
2,3,Wheat,-,NaN,NaN,Nakuru Wakulima,65.00,NaN,NaN,Nakuru,2024-03-08
3,4,Wheat,-,NaN,NaN,Daraja Mbili,60.00,80.0,250.0,Kisii,2024-03-08
4,5,Wheat,-,NaN,NaN,Maua,77.78,100.0,1980.0,Meru,2024-03-07


In [15]:
# Convert the 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Check the result
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                2000 non-null   int64         
 1   commodity         2000 non-null   object        
 2   classification    2000 non-null   object        
 3   grade             0 non-null      object        
 4   sex               0 non-null      object        
 5   market            2000 non-null   object        
 6   wholesale_per_kg  1730 non-null   float64       
 7   retail_per_kg     1586 non-null   float64       
 8   supply_volume     1158 non-null   float64       
 9   county            2000 non-null   object        
 10  date              2000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(6)
memory usage: 172.0+ KB
None


,id,commodity,classification,grade,sex,market,wholesale_per_kg,retail_per_kg,supply_volume,county,date
0,1,Wheat,-,NaN,NaN,Kibiok,75.00,80.0,450.0,Nandi,2024-03-10
1,2,Wheat,-,NaN,NaN,Kabiyet Market,100.00,130.0,1000.0,Nandi,2024-03-08
2,3,Wheat,-,NaN,NaN,Nakuru Wakulima,65.00,NaN,NaN,Nakuru,2024-03-08
3,4,Wheat,-,NaN,NaN,Daraja Mbili,60.00,80.0,250.0,Kisii,2024-03-08
4,5,Wheat,-,NaN,NaN,Maua,77.78,100.0,1980.0,Meru,2024-03-07


In [16]:
#Check for any empty columns in the dataset
empty_cols = df.columns[df.isna().all()]
print(empty_cols)

Index(['grade', 'sex'], dtype='object')


In [17]:
#Remove empty columns from dataframe
df = df.dropna(axis=1, how='all')
df.head()

,id,commodity,classification,market,wholesale_per_kg,retail_per_kg,supply_volume,county,date
0,1,Wheat,-,Kibiok,75.00,80.0,450.0,Nandi,2024-03-10
1,2,Wheat,-,Kabiyet Market,100.00,130.0,1000.0,Nandi,2024-03-08
2,3,Wheat,-,Nakuru Wakulima,65.00,NaN,NaN,Nakuru,2024-03-08
3,4,Wheat,-,Daraja Mbili,60.00,80.0,250.0,Kisii,2024-03-08
4,5,Wheat,-,Maua,77.78,100.0,1980.0,Meru,2024-03-07


In [18]:
cleaned_df = df
cleaned_df.head()

,id,commodity,classification,market,wholesale_per_kg,retail_per_kg,supply_volume,county,date
0,1,Wheat,-,Kibiok,75.00,80.0,450.0,Nandi,2024-03-10
1,2,Wheat,-,Kabiyet Market,100.00,130.0,1000.0,Nandi,2024-03-08
2,3,Wheat,-,Nakuru Wakulima,65.00,NaN,NaN,Nakuru,2024-03-08
3,4,Wheat,-,Daraja Mbili,60.00,80.0,250.0,Kisii,2024-03-08
4,5,Wheat,-,Maua,77.78,100.0,1980.0,Meru,2024-03-07


In [ ]:
# Establish a connection to my new supebase database
# Load environment variables from the .env file
load_dotenv()

# Access the credentials for new database
new_db_host = os.getenv('cleaned_host')
new_db_port = os.getenv('cleaned_port')
new_database_name = os.getenv('cleaned_database')
new_db_username = os.getenv('cleaned_username')
new_db_password = os.getenv('cleaned_password')


supabase_engine = create_engine(f'postgresql+psycopg2://{new_db_username}:{new_db_password}@{new_db_host}:{new_db_port}/{new_database_name}',
                               future=True  # Ensures SQLAlchemy behaves like 2.0
                                )


Host: db.qdatczbbamscpkrjsayo.supabase.co, Port: 5432, Database: postgres, Username: postgres, Password: 7RR0BVadHyiT8CKd


In [22]:
#Load the cleaned data
cleaned_df.to_sql('cleaned_agric_data', con=supabase_engine, if_exists='replace', index=False)

print("Upload successful!")

Upload successful!
